Import Problems:

* value of columns needs nice parsing
* 2 header lines?
* What about time ref values in columns and in rows
* Grouping if disctrict code happens to be multiple times in it

In [73]:
import requests
import pandas as pd
import json
import os
import IPython
import random
import csv
import re
import chardet
#injected variables
#url='http://www.wien.gv.at/statistik/ogd/vie_403.csv'
url='http://www.wien.gv.at/politik/wahlen/ogd/bv051_99999999_9999_spr.csv'
#url='http://service.stmk.gv.at/ogd/OGD_Data_ABT17/statistik/BEV_GESCHL/STMK_01012016_SEX.csv'
name='testing'
description='desc'
publisher='bla'
portal="https"
#data reading and preparation
def readandcleancsv( url ):
    r = requests.get(url)
    filename=str(random.getrandbits(64))+".csv.tmp"
    header =[];
    rows=[];
    encoding='utf-8';
    with open(filename, "wb") as code:
        code.write(r.content)
    with open(filename, 'rb') as f:
        encoding = chardet.detect(f.read())['encoding']
    with open(filename, 'r', encoding=encoding) as csvfile:        
        try:
            dialect = csv.Sniffer().sniff(csvfile.read(1024*16), delimiters=';,\t')
        except Exception as e:
            print("exception"+e)
        csvfile.seek(0)
        reader = csv.reader(csvfile, dialect)
        #go through all lines
        #Todo get the header
        #Todo detect dates and get them in a fixed format
        #Todo get Numeric Values and get them in a fixed format  
        #empty string
        for index, entry in enumerate(reader):
            #test for header in first 2 lines
            if( (index==0 or index==1) and len(header) == 0):
                empties = 0;
                for i in entry:
                    if(i==""):
                        empties+=1
                if(empties <= 1):
                    #print("using header at index: "+str(index))
                    header=entry
            else:
                rows.append(entry)
    #amount of rows
    #print(len(rows))
    #amount of columns in header
    #print(len(header))
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(header);
        for element in rows:
            writer.writerow(element);
    return filename
filename = readandcleancsv( url )
#print(filename)
df = pd.read_csv(filename,sep=';', thousands='.', decimal=',')
#test if SUB_DISTRICT_CODE exists
districtCode =""
if 'DISTRICT_CODE' in df.columns:
    districtCode="DISTRICT_CODE"
if 'SUB_DISTRICT_CODE' in df.columns:
    districtCode='SUB_DISTRICT_CODE'
if 'LAU_CODE' in df.columns:
    districtCode='LAU_CODE'
df = df.drop_duplicates(subset=districtCode)
df.columns=df.columns.str.replace('#','')
df.columns=df.columns.str.replace('.','')
df.columns=df.columns.str.replace(':','')
df.columns=df.columns.str.replace('"','')
df.columns=df.columns.str.replace(' ','')
numeric_columnlist = list(df._get_numeric_data().columns)
#filter out empty column names
string_columns=[item for item in list(df.columns) if item not in numeric_columnlist and item!='']
numeric_columnlist=[item for item in numeric_columnlist if item!='' and item!='DISTRICT_CODE' and item!='SUB_DISTRICT_CODE' and item!='LAU_CODE']
#last numeric field as entity field for know.
valueField = random.choice(numeric_columnlist)
entityField = string_columns[-1]
data_dict = df.to_dict(orient='records')
#delete temp file
os.remove(filename)
requestOBJ = {
        "data" : data_dict,
        "meta": {
            "name": name + " "+valueField,
            "sourceOrganization": publisher,
            "source": publisher,
            "description": description,
            "publisher_name": publisher,
            "publisher_homepage": portal,
            "publisher_contact": "EMAIL ADRESS OF PUBLISHER",
            "publisher_tags": ["City of Vienna", "Demographie", "Population"],
            "accessUrl": "/api/data/",
            "accessFormat": "json",
            "frequency": "INSERT EXPECTED DATA UPDATE CYCLE",
            "license": "Creative Commons Namensnennung 3.0 Österreich",
            "citation": "INSERT CITATION IF AVAILABLE",
            "isoField": districtCode,
            "entityField": entityField,
            "timeField": "null",
            "timeDimension": "false",
            "timeUnit": "year",
            "valueField": valueField,
            "unit": "Value:",
            "colors": ["#ffc971", "#ffb627", "#ff9505", "#e2711d", "#cc5803"],
            "legendtitles": ["low", "med-low", "med", "med-high", "high"],
            "tooltip": [{"label": valueField, "field": "data:"+valueField }],
            "theme": "light"
        },
        "parameters": {
            "source": "manual",
            "provider": "reboting",
            "layer": "choropleth"
        }
    }
#print(json.dumps(requestOBJ, indent=3, sort_keys=True))
r = requests.post("http://52.166.116.205:2301/manual", json=requestOBJ)
url = 'https://doh.23degrees.io/viz/'+r.json()['slug']
iframe= '<div class="intrinsic-container" style="position: relative; padding-bottom: 56.25%;height: 0; overflow: hidden; width: 100%;height: auto;"><iframe src="' + url + '" allowfullscreen style=" position: absolute; top: 0; left: 0; width: 100%; height: 100%; "></iframe></div>'
#print(url)
IPython.display.HTML(iframe)

In [65]:
import chardet

In [2]:
!pip install git+git://github.com/ODInfoBiz/pyyacp

  Cloning git://github.com/ODInfoBiz/pyyacp to /tmp/pip-VgbbfU-build
  Using cached structlog-17.2.0-py2.py3-none-any.whl
  Using cached unicodecsv-0.14.1.tar.gz
  Using cached faststat-0.3.1.zip
  Running setup.py bdist_wheel for unicodecsv ... - done
  Stored in directory: /home/jovyan/.cache/pip/wheels/97/e2/16/219fa93b83edaff912b6805cfa19d0597e21f8d353f3e2d22f
  Running setup.py bdist_wheel for faststat ... - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b0/86/98/b8156408d47f75f455bacb6e809796627a0b412c64e9f8414a
Successfully built unicodecsv faststat
  Running setup.py install for pyyacp ... - done


In [3]:
!pip install git+git://github.com/sebneu/anycsv

  Cloning git://github.com/sebneu/anycsv to /tmp/pip-PbjBoY-build
  Using cached file-magic-0.3.0.tar.gz
    100% |████████████████████████████████| 204kB 2.0MB/s 
  Running setup.py bdist_wheel for file-magic ... - done
  Stored in directory: /home/jovyan/.cache/pip/wheels/cb/e1/b4/516b540f7252a90672c02d6f184f1c8f768f3a32be29d798b5
Successfully built file-magic
  Running setup.py install for anycsv ... - done


In [3]:
!git clone https://github.com/sebneu/anycsv.git

Cloning into 'anycsv'...
remote: Counting objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175
Receiving objects: 100% (175/175), 33.83 KiB | 0 bytes/s, done.
Resolving deltas: 100% (110/110), done.
Checking connectivity... done.


In [5]:
!cd anycsv;python setup.py install;

running install
running bdist_egg
running egg_info
creating anycsv.egg-info
writing anycsv.egg-info/PKG-INFO
writing dependency_links to anycsv.egg-info/dependency_links.txt
writing requirements to anycsv.egg-info/requires.txt
writing top-level names to anycsv.egg-info/top_level.txt
writing manifest file 'anycsv.egg-info/SOURCES.txt'
reading manifest file 'anycsv.egg-info/SOURCES.txt'
writing manifest file 'anycsv.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/anycsv
copying anycsv/io_tools.py -> build/lib/anycsv
copying anycsv/exceptions.py -> build/lib/anycsv
copying anycsv/csv_parser.py -> build/lib/anycsv
copying anycsv/__init__.py -> build/lib/anycsv
copying anycsv/csv_model.py -> build/lib/anycsv
copying anycsv/csv.py -> build/lib/anycsv
copying anycsv/dialect.py -> build/lib/anycsv
copying anycsv/encoding.py -> build/lib/anycsv
creating build/bdist.linux-x86_6

In [25]:
df[numeric_columnlist].idxmax(axis=1, skipna=True)

,T,WV,WK,BZ,SPR,WBER,ABG,UNG,SPOE,FPOE,...,WWW,GS,PRO,KEINE,OKKI,EMP,PB,OE,AL,WIF
0,4,1,1,1,0,0,208,5,60,15,...,0,0,0,0,0,0,0,0,0,0
20,4,1,2,2,0,0,607,15,327,78,...,0,1,0,0,0,0,0,0,0,0
133,4,1,3,3,0,0,561,16,254,53,...,0,0,0,0,0,0,0,0,0,0
251,4,1,4,4,0,0,246,5,82,13,...,3,0,0,0,0,0,0,0,0,0
286,4,1,5,5,0,0,331,11,161,43,...,0,0,0,0,0,0,0,0,0,0
345,4,1,6,6,0,0,234,6,101,19,...,0,0,0,0,0,0,0,0,0,0
381,4,1,7,7,0,0,244,2,76,19,...,0,0,0,0,0,0,0,0,0,0
413,4,1,8,8,0,0,186,6,67,13,...,0,0,44,0,0,0,0,0,0,0
439,4,1,9,9,0,0,242,5,103,19,...,0,0,0,3,0,0,0,0,0,0
487,4,1,10,10,0,0,905,20,606,96,...,0,0,0,0,0,0,0,0,0,0


In [9]:
string_columns

['NUTS1', 'NUTS2', 'NUTS3', '']

In [3]:
df.describe()

,DISTRICT_CODE,T,WV,WK,BZ,SPR,WBER,ABG,UNG,SPOE,...,WWW,GS,PRO,KEINE,OKKI,EMP,PB,OE,AL,WIF
count,23.000000,23.0,23.0,23.00000,23.00000,23.0,23.0,23.000000,23.000000,23.000000,...,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.0,23.000000,23.000000
mean,91201.000000,4.0,1.0,12.00000,12.00000,0.0,0.0,480.652174,10.956522,258.608696,...,0.130435,0.043478,1.913043,0.130435,0.130435,0.043478,0.130435,0.0,0.043478,0.173913
std,678.232998,0.0,0.0,6.78233,6.78233,0.0,0.0,215.182715,4.476994,159.755990,...,0.625543,0.208514,9.174634,0.625543,0.625543,0.208514,0.625543,0.0,0.208514,0.650327
min,90101.000000,4.0,1.0,1.00000,1.00000,0.0,0.0,186.000000,2.000000,60.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,90651.000000,4.0,1.0,6.50000,6.50000,0.0,0.0,288.500000,8.000000,123.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,91201.000000,4.0,1.0,12.00000,12.00000,0.0,0.0,519.000000,11.000000,243.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,91751.000000,4.0,1.0,17.50000,17.50000,0.0,0.0,599.500000,14.500000,341.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
max,92301.000000,4.0,1.0,23.00000,23.00000,0.0,0.0,905.000000,20.000000,606.000000,...,3.000000,1.000000,44.000000,3.000000,3.000000,1.000000,3.000000,0.0,1.000000,3.000000


In [96]:
df.describe(include="all")

,NUTS1,NUTS2,NUTS3,DISTRICT_CODE,T,WV,WK,BZ,SPR,WBER,...,GS,PRO,KEINE,OKKI,EMP,PB,OE,AL,WIF,
count,23,23,23,23.000000,23.0,23.0,23.00000,23.00000,23.0,23.0,...,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.0,23.000000,23.000000,23
unique,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
top,AT1,AT13,AT130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
freq,23,23,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
mean,NaN,NaN,NaN,91201.000000,4.0,1.0,12.00000,12.00000,0.0,0.0,...,0.043478,1.913043,0.130435,0.130435,0.043478,0.130435,0.0,0.043478,0.173913,NaN
std,NaN,NaN,NaN,678.232998,0.0,0.0,6.78233,6.78233,0.0,0.0,...,0.208514,9.174634,0.625543,0.625543,0.208514,0.625543,0.0,0.208514,0.650327,NaN
min,NaN,NaN,NaN,90101.000000,4.0,1.0,1.00000,1.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,90651.000000,4.0,1.0,6.50000,6.50000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN
50%,NaN,NaN,NaN,91201.000000,4.0,1.0,12.00000,12.00000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN
75%,NaN,NaN,NaN,91751.000000,4.0,1.0,17.50000,17.50000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,NaN
